## Goal: Compute an Event-Related Potential (ERP)

Today we will analyze real EEG data to compute an **Event-Related Potential (ERP)**, a classic way to study how the brain responds to specific events.
You will take continuous EEG data, extract time-locked trials, and average them to reveal a clear neural response.


### What is an ERP?

When a person sees or reacts to a stimulus, the brain produces small, time-locked electrical responses.
In a single trial, these responses are buried in ongoing brain activity, so they are hard to see.
If we repeat the same kind of event many times and **average** the EEG segments time-locked to that event, the random noise cancels out and the consistent signal remains.
That averaged signal — showing voltage over time relative to the event — is the **Event-Related Potential** (ERP).


### MNE-Python: our tool for EEG analysis

We will use **MNE-Python**, a powerful and well-structured library for EEG and MEG analysis.
It builds directly on **NumPy** arrays, so under the hood, all the data are numbers in standard array form — but MNE adds rich metadata and many built-in functions that handle the tricky parts of EEG analysis for us.

Some of what MNE manages automatically:

* consistent handling of channel information, sampling rate, and units
* correct alignment of data and event timings
* referencing, filtering, baseline correction, and visualization tools
* efficient storage and loading of large datasets

You can (and should) look up the most important MNE classes and functions here:
[https://mne.tools/stable/api/most_used_classes.html](https://mne.tools/stable/api/most_used_classes.html)


### Data types in MNE

MNE organizes data into a few key **data classes**, each representing a stage of analysis.
They can “stem from” each other — meaning you start with continuous data, then segment it, then average it.

| Stage            | MNE class    | Description                                                                                         |
| ---------------- | ------------ | --------------------------------------------------------------------------------------------------- |
| Continuous data  | `mne.io.Raw` | The full continuous EEG recording. Contains all channels and the time series for the whole session. |
| Trial-based data | `mne.Epochs` | Segments (short time windows) cut out from the Raw data around each event — one per trial.          |
| Averaged data    | `mne.Evoked` | The average of multiple epochs belonging to the same condition — the ERP.                           |

The data always move through this sequence:

```
Raw (continuous)  →  Epochs (trials)  →  Evoked (averaged ERP)    
```

Each object carries an `.info` attribute that stores all metadata (channels, sampling frequency, etc.), so this information travels with the data at every step.

---

### Steps you’ll perform today

Below, you will go through these steps one by one, with explanations and guiding questions:

1. **Inspect the data**

   * Understand what an `mne.io.Raw` object contains.
   * Look at the recording’s metadata (`info`), sampling rate, and channels.

2. **Select one channel**

   * Focus on one EEG electrode (e.g., “Fz”) to make the analysis simpler.
   * Drop all other channels and observe how the data shape changes.

3. **Apply a light filter**

   * Filter out slow drifts and high-frequency noise to clean the signal a bit.
   * (We’ll use a broad 0.1–30 Hz filter.)

4. **Inspect the events**

   * The dataset includes event markers showing when stimuli occurred.
   * You’ll extract and visualize these to see how many trials and conditions exist.

5. **Epoch the data**

   * Cut the continuous EEG into short time windows (epochs) around each event.
   * Each epoch corresponds to one trial (e.g., one Flanker stimulus).

6. **Average to get ERPs**

   * Average the epochs within each condition to create the final ERPs.
   * The result is an `mne.Evoked` object, which you can plot and compare across conditions.

6. **Combine ERPs of different conditions**

   * Average the epochs within each condition to create the final ERPs.
   * The result is an `mne.Evoked` object, which you can plot and compare across conditions.

7. **Average to get ERPs**
   * Visualize the ERPs

By the end of today’s exercise, you’ll understand how continuous EEG becomes trial-based data, how trials become averaged ERPs, and how MNE-Python manages these transformations cleanly and efficiently.


In [ ]:
!pip install mne # We install mne, a library for the analysis of EEG, MEG, and related signals
import mne
import matplotlib.pyplot as plt
import numpy as np

# ERP CORE Dataset

MNE is a python library that supports a wide range of EEG analyses. It also ships with various example datasets, one of which is the so-called mne ERP CORE. 
Derived from a larger dataset, it contains a single subject during a classical task of cognitive neuroscience, the so-called Flanker task.

## Flanker task

During a flanker task, subjects are presented with stimuli that may look like this

'> > > < > > >'                    (incongruent/incompatible condition)  
       

'< < < < < < <'                    (congruent/compatible condition)  
       

       
The task is to focus on the middle symbol, here <, and to ignore the surrounding symbols. Subjects are asked to response as fast as possible with the left or right arrow, depending on the stimulus in the center. This task is easier when all symbols are the same (congruent condition), and harder when the surrounding symbols point into the other direction (incongruent condition).

## Cognitive events during the Flanker task
When executing this task, multiple events occur, e.g.  
  
(1) Preparation for Stimulus Presentation  
(2) Visual Perception of the stimulus  
(3) Evaluation of the stimulus  
(4) Preparation of motor response  
(5) Execution of motor response  
(6) Evaluation of response - was my response correct or incorrect?  


In [ ]:
# Loading the Data

# We define the data path and the file
data_dir = mne.datasets.erp_core.data_path()
infile = data_dir / "ERP-CORE_Subject-001_Task-Flankers_eeg.fif"

# We load the EEG data as a so-called RawArray; a specific class of mne
raw = mne.io.read_raw(infile, preload=True)

# We filter the data to be between 0.1 and 40 Hz, which is a common range for EEG
raw.notch_filter(50)
raw.filter(l_freq=0.1, h_freq=30)


In [ ]:
# Here, we havea look at what the data approximately looks like
# What do you see in the plot?
raw.plot(start=60, duration = 2)

In [ ]:
# Pick a channel
# for now, we want to select a single channel for our analyses,  good choice could be Fz
analysis_channel = ""
raw = raw.pick(analysis_channel)

# Epoching Data

ERPs are computed across multiple segments of data. As the raw data so far is continuous, we must segment it around certain events (e.g., stimulus presentation) in our data.
In EEG research, a segment relating to one trial is usually called an __epoch__.
Relative to the event, an epoch has a start (e.g., 200ms before event onset) and an end (e.g., 1000ms after event onset). Often, we want to subtract also mean pre-stimulus activity from epochs, so that they all share the same baseline.

When epoching, our data changes from a 2D array (channels x time) to a 3D array (channels x epoch number x time).
For all these steps, mne python has fantastic tools on board.

## (1) Extracting events from raw data
To construct epochs, we must first find out which types of events were recorded along side our data. You can get the events from the raw data using
``` mne.events_from_annotations(raw) ```. Please explore the output of this function, and make sure you understand it. If something is unclear, you can refer to the mne documentation.

In [ ]:
# Please have a look at the mne documentation to understand the output of events_from_annotations.
all_events, all_event_id = mne.events_from_annotations(raw)

In [ ]:
# Here, explore both 'all_events' and 'all_event_id'

## Check yourself

Do you understand what the different columns (especially the first and the last column) in 'all_events' relate to? This is really important for the next steps and you shouldn't continue otherwise.

## (2) Constructing Epochs

### Filtering events
We see that there are two classes of stimuli in the Flanker task: compatible, and incompatible stimuli. We now want to create 2 different epoch objects. One should capture all compatible, and the other all incompatible stimulus presentations.

You can use your numpy skills to filter ```all_events``` to generate 2 matrices that contain only the compatible, and the incompatible events, respectively.


In [ ]:
# Get the events (from all_events) that relate only to the compatible stimuli
# Hint: You can use np.isin for this! 

# ... fill out the rest
compatible_ids = []


In [ ]:
# Repeat this for incompatible stimuli


### Creating Epochs objects

To create epochs, you can use the ```mne.Epochs()``` constructor. Please make one epochs project for compatible and one for incompatible conditions.

In [ ]:
# Build the Epochs for compatible and incompatible here

### Evoked from Epochs

Epochs contain still all the individual trials. To construct an event related potential, we need to average across the epochs. In mne, such an average epochs object is called an ```Evoked```.

In MNE this is very easy. 
```your_evoked = your_epoch_object.average()```

In [ ]:
# Please generate Evokeds for both conditions

### Plotting Epochs

Evokeds in mne can be plotted easily. Just use ```your_evoked.plot()``` 

In [ ]:
# Please plot the Evokeds for both conditions

### Difference Waves

Compatible and Incompatible are two different conditions that relate to the processing difficulty of a stimulus. In the incompatible condition, the correct stimulus orientation is more difficult to discern, while this is easier in the compatible condition.

To generate a difference wave, we have to use some of mne's advanced machinery, namely:  
```mne.combine_evoked([evoked1, evoked2], weights = [weight1, weight2])```  
This functiona allows you to generate a (weighted) difference between different evoked arrays.

Please have a look at the mne documentation on how to use this function exactly.